# Research question 3

## Graph generation
In this first section, we extract the RTs graph from our tweets data.

In [ ]:
#Download required files stored in Google Drive
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1BNgqe8vJsmAw0Y09wfILyEn7OzNDrz8o' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1BNgqe8vJsmAw0Y09wfILyEn7OzNDrz8o" -O tweets.json && rm -rf /tmp/cookies.txt

In [ ]:
# Read json file with tweets
import os

with open(os.path.join('tweets.json'), "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_records(data)

In [ ]:
df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,possibly_sensitive,display_text_range,extended_tweet,extended_entities,withheld_in_countries
0,Sat Nov 14 09:55:00 +0000 2020,1327550590667386880,1327550590667386880,RT @LLinWood: Many of you are weary of the lie...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 1061507476649058304, 'id_str': '1061507...",None,None,None,None,{'created_at': 'Sat Nov 14 06:03:53 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sat Nov 14 09:55:00 +0000 2020,1327550590566674432,1327550590566674432,Amen 🇺🇸,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 2229194696, 'id_str': '2229194696', 'na...",None,None,None,None,NaN,True,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700145,1.327492e+18,1327492427951841282,{'created_at': 'Sat Nov 14 06:03:53 +0000 2020...,"{'url': 'https://t.co/6q7fDTZIDl', 'expanded':...",NaN,NaN,NaN,NaN,NaN
2,Sat Nov 14 09:55:00 +0000 2020,1327550590826770432,1327550590826770432,RT @elonmusk: Something extremely bogus is goi...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 1321676820673159172, 'id_str': '1321676...",None,None,None,None,{'created_at': 'Fri Nov 13 05:47:11 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sat Nov 14 09:55:00 +0000 2020,1327550590746906624,1327550590746906624,RT @Msdesignerlady: 📢 Trump supporters are pla...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 1537352035, 'id_str': '1537352035', 'na...",None,None,None,None,{'created_at': 'Fri Nov 13 16:33:43 +0000 2020...,True,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700188,1.326851e+18,1326850908957908992,{'created_at': 'Thu Nov 12 11:34:43 +0000 2020...,"{'url': 'https://t.co/FU0zy1I26h', 'expanded':...",NaN,NaN,NaN,NaN,NaN
4,Sat Nov 14 09:55:00 +0000 2020,1327550590960889858,1327550590960889858,Inside the hunt for a Covid-19 vaccine: how Bi...,"<a href=""https://about.twitter.com/products/tw...",False,NaN,None,NaN,None,None,"{'id': 18571616, 'id_str': '18571616', 'name':...",None,None,None,None,NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1605347700239,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(121738, 37)

In [ ]:
def get_retweet_info(row):
    """
    Returns information from a RT. It will output who is retweeting (from), user who posted the original tweet (to)
    and the number of followers from the original user (followers_to).
    """
    if str(row['retweeted_status']) != 'nan':
        rt = row['retweeted_status']
        origin = row['user']

        from_ = origin['id_str']
        to_ = rt['user']['id_str']
        followers_to = rt['user']['followers_count']
        return from_, to_, followers_to

    else: # For non-rts
        return None, None, None

In [ ]:
# Generate new summary columns for our dataframe with RT information
df['from'], df['to'], df['followers_to'] = zip(*df.apply(lambda x: get_retweet_info(x), axis=1))

In [ ]:
df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,possibly_sensitive,display_text_range,extended_tweet,extended_entities,withheld_in_countries,from,to,followers_to
0,Sat Nov 14 09:55:00 +0000 2020,1327550590667386880,1327550590667386880,RT @LLinWood: Many of you are weary of the lie...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 1061507476649058304, 'id_str': '1061507...",None,None,None,None,{'created_at': 'Sat Nov 14 06:03:53 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1061507476649058304,187680645,319942
1,Sat Nov 14 09:55:00 +0000 2020,1327550590566674432,1327550590566674432,Amen 🇺🇸,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 2229194696, 'id_str': '2229194696', 'na...",None,None,None,None,NaN,True,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700145,1.327492e+18,1327492427951841282,{'created_at': 'Sat Nov 14 06:03:53 +0000 2020...,"{'url': 'https://t.co/6q7fDTZIDl', 'expanded':...",NaN,NaN,NaN,NaN,NaN,None,None,None
2,Sat Nov 14 09:55:00 +0000 2020,1327550590826770432,1327550590826770432,RT @elonmusk: Something extremely bogus is goi...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 1321676820673159172, 'id_str': '1321676...",None,None,None,None,{'created_at': 'Fri Nov 13 05:47:11 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1321676820673159172,44196397,39991414
3,Sat Nov 14 09:55:00 +0000 2020,1327550590746906624,1327550590746906624,RT @Msdesignerlady: 📢 Trump supporters are pla...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 1537352035, 'id_str': '1537352035', 'na...",None,None,None,None,{'created_at': 'Fri Nov 13 16:33:43 +0000 2020...,True,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700188,1.326851e+18,1326850908957908992,{'created_at': 'Thu Nov 12 11:34:43 +0000 2020...,"{'url': 'https://t.co/FU0zy1I26h', 'expanded':...",NaN,NaN,NaN,NaN,NaN,1537352035,32467167,16043
4,Sat Nov 14 09:55:00 +0000 2020,1327550590960889858,1327550590960889858,Inside the hunt for a Covid-19 vaccine: how Bi...,"<a href=""https://about.twitter.com/products/tw...",False,NaN,None,NaN,None,None,"{'id': 18571616, 'id_str': '18571616', 'name':...",None,None,None,None,NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1605347700239,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,None,None,None


In [ ]:
# We filter only those tweets which are RTs
df_rts = df[df['retweeted_status'].notnull()]

In [ ]:
df_rts.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,possibly_sensitive,display_text_range,extended_tweet,extended_entities,withheld_in_countries,from,to,followers_to
0,Sat Nov 14 09:55:00 +0000 2020,1327550590667386880,1327550590667386880,RT @LLinWood: Many of you are weary of the lie...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 1061507476649058304, 'id_str': '1061507...",None,None,None,None,{'created_at': 'Sat Nov 14 06:03:53 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1061507476649058304,187680645,319942
2,Sat Nov 14 09:55:00 +0000 2020,1327550590826770432,1327550590826770432,RT @elonmusk: Something extremely bogus is goi...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 1321676820673159172, 'id_str': '1321676...",None,None,None,None,{'created_at': 'Fri Nov 13 05:47:11 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1321676820673159172,44196397,39991414
3,Sat Nov 14 09:55:00 +0000 2020,1327550590746906624,1327550590746906624,RT @Msdesignerlady: 📢 Trump supporters are pla...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 1537352035, 'id_str': '1537352035', 'na...",None,None,None,None,{'created_at': 'Fri Nov 13 16:33:43 +0000 2020...,True,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700188,1.326851e+18,1326850908957908992,{'created_at': 'Thu Nov 12 11:34:43 +0000 2020...,"{'url': 'https://t.co/FU0zy1I26h', 'expanded':...",NaN,NaN,NaN,NaN,NaN,1537352035,32467167,16043
6,Sat Nov 14 09:55:00 +0000 2020,1327550592403828737,1327550592403828737,RT @tropicanapussy: states trying to stop the ...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 511759357, 'id_str': '511759357', 'name...",None,None,None,None,{'created_at': 'Fri Nov 13 05:41:27 +0000 2020...,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605347700583,NaN,NaN,NaN,NaN,False,NaN,NaN,"{'media': [{'id': 1327124391939989505, 'id_str...",NaN,511759357,195503724,24790
7,Sat Nov 14 09:55:00 +0000 2020,1327550592533852162,1327550592533852162,RT @NHSEnglandLDN: To Londoners celebrating #D...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 1563827293, 'id_str': '1563827293', 'na...",None,None,None,None,{'created_at': 'Sat Nov 14 08:05:03 +0000 2020...,False,0,0,0,0,"{'hashtags': [{'text': 'Diwali', 'indices': [4...",False,False,low,en,1605347700614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1563827293,31129844,98110


In [ ]:
# We obtain the number of followers for each destination user (we keep the maximum number in our dataset)
followers = df_rts.groupby('to')['followers_to'].apply(max).to_dict()

In [ ]:
followers

{'1000171845264359426': 1682,
 '1000201182021730304': 23201,
 '1000323887479967744': 204,
 '1000499544': 8344,
 '1000768243': 138,
 '1001130590442143744': 478,
 '1001285146392383489': 263224,
 '1001308036051456000': 954,
 '100136358': 140941,
 '1001454158682165249': 1227,
 '1001918140924813314': 3550,
 '1002084811': 19740,
 '1002117661344452609': 368,
 '1002456116532375552': 70009,
 '1002645371942629377': 4065,
 '1002645771563487233': 912,
 '1003140853571862528': 69897,
 '1003348881529364482': 3288,
 '1003753267044597760': 320,
 '1003882790': 3476,
 '1003904023370567682': 334,
 '1004022175': 7542,
 '1004040782': 3290,
 '1004129663420174339': 111,
 '1004523129371230208': 13944,
 '1004683561': 3588,
 '1004694289': 38278,
 '1004792929804783621': 11,
 '1004938829915013120': 6005,
 '100493922': 19424,
 '1005025809789280256': 1809,
 '1005224977430241280': 483,
 '1005285779822673920': 51974,
 '100539502': 693,
 '1005417075441950720': 588,
 '1005557387376279552': 80170,
 '1005770840': 24343,
 

In [ ]:
# We obtain the network. We obtain a dictionary where each source node is assigned a list of destination users.
edges = df_rts.groupby('from')['to'].apply(list).to_dict()

In [ ]:
edges

{'1000017561902370818': ['631692563'],
 '1000044230801350657': ['618387275'],
 '1000050544617107456': ['44196397'],
 '1000094942': ['631692563'],
 '1000099433936244737': ['631692563'],
 '10001212': ['2735591'],
 '1000126926932000768': ['8128992'],
 '100014940': ['2899459274'],
 '1000162987406114817': ['946285781265600512'],
 '1000196034419621888': ['946285781265600512'],
 '1000242837852688385': ['3225450034'],
 '1000307556550668288': ['631692563'],
 '100032683': ['562869150', '939091'],
 '1000374049376219138': ['1923844412'],
 '1000380653358239745': ['1149241366314192898',
  '1192835328060674048',
  '187680645',
  '16834659'],
 '1000411364790222850': ['4639413136'],
 '1000447027061325824': ['2312893002'],
 '1000449838989873152': ['1138814255854247941'],
 '1000465056780480518': ['1050837690940252161',
  '64608304',
  '967004083470131200',
  '286998245'],
 '1000529617948676097': ['1065248841212010496'],
 '1000532220761853952': ['122533830'],
 '1000565834425798656': ['1252831932552904705'

In [ ]:
len(edges)

62259

In [ ]:
# Store the obtained data for later usage
import json
with open('graph_edges.json', 'w') as fp:
    json.dump(edges, fp)

with open('followers.json', 'w') as fp:
    json.dump(followers, fp)

## Graph analysis
Using the previously generated data, we can now perform edge prediction in the network.

In [191]:
# Load previously generated data
import json

with open('graph_edges.json', 'r') as f:
    network = json.load(f)

with open('followers.json', 'r') as f:
    followers = json.load(f)

In [220]:
# Remove duplicated edges in our retweet network because a user may have retweeted the same user several times
for source in network:
    network[source] = list(set(network[source]))

In [221]:
def map_network(network):
    """
    Maps each node name to an integer ranging from 0 to the number of nodes and returns the mapping
    """
    mapping = {}
    new_value = 0

    for key, value in network.items():
        if key not in mapping:
            mapping[key] = new_value
            new_value+=1
        for v in value:
            if v not in mapping:
                mapping[v] = new_value
                new_value+=1

    return mapping, new_value

In [222]:
# We map our users
mapping, nodes = map_network(network)

In [6]:
!pip install python-igraph==0.8.3

In [7]:
import igraph

In [223]:
# We generate an undirected graph with our nodes and their retweets connections using the mapping
graph = igraph.Graph(directed=False)
graph.add_vertices(nodes)
graph.add_edges([(mapping[v], mapping[a]) for v in network.keys() for a in network[v]])
graph.is_directed()

False

In [224]:
# Obtain weights for our weighted graph (used for the last section) from followers count
w_graph = graph.copy()
weights = []
for source in network.keys():
    for dest in network[source]:
        weights.append(followers[dest])

In [225]:
# Assign the weights to the graph
w_graph.es['weight'] = weights

#### Test set generation
We sample 20% of the edges as test set

In [302]:
import random

test_size = 0.2

# Total number of edges
N = len(graph.es)

# Id for all edges in the graph
all_idxs = range(N)

# Randomly select test edges
test_idxs = np.random.choice(all_idxs, size=int(test_size*N), replace=False)

In [303]:
print("Original number of edges: {}".format(N))
print("Testing number of edges: {}".format(len(test_idxs)))

Original number of edges: 80869
Testing number of edges: 16173


In [314]:
# Training graphs generation

# We delete test edges from the unweighted graph
training_graph = graph.copy()
training_graph.delete_edges(test_idxs)

# We delete test edges from the weighted graph
w_training_graph = w_graph.copy()
w_training_graph.delete_edges(test_idxs)

In [315]:
# We will store our test set as groundtruth for the evaluation.
# Source nodes from the test set are kept to perform evaluation. We only keep source nodes to reduce the size of the problem

ground_truth = set()
source_nodes = set() # Source nodes for prediction

for idx, edge in enumerate(graph.es):
    if idx in test_idxs:
        n1 = edge.source
        n2 = edge.target
        ground_truth.add((n1, n2)) # Store as groundtruth
        source_nodes.add(n1) # Store source nodes

In [316]:
print("Our groundtruth is composed by {} edges".format(len(ground_truth)))
print("We will generate recommendations for {} edges".format(len(source_nodes)))

Our groundtruth is composed by 16172 edges
We will generate recommendations for 4865 edges


In [317]:
# Compute nodes at distance 2 from nodes in the test set
def find_nodes_at_distance_2(graph, nodes):
    """
    Starting from a graph this function returns all the nodes at distance 2
    from the selected nodes. Returns the set of possible edges and a dictionary with the potential recommendations
    for each source node.
    """

    set_recommendations = set()
    dict_recommendations = {}
    
    for n1 in graph.vs:

        if n1.index in nodes:
        
            # all the nodes at distance 1
            nodes_at_most_distant_1 = set(graph.neighborhood(n1, order=1))

            # all the nodes at distance 1 and distance 2

            nodes_at_most_distant_2 = set(graph.neighborhood(n1, order=2))
          
            # only the nodes at distance 2
            only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1

            dict_recommendations[n1.index] = []        

            # since n1 is an igraph vertex object, we need to extract the id
            n1_index = n1.index
                    
            # check if empty set
            if len(only_nodes_at_distance_2) > 0:

                for n2 in only_nodes_at_distance_2:
                  
                    set_recommendations.add((n1_index, n2))
                    dict_recommendations[n1.index].append(n2)

    return set_recommendations, dict_recommendations

In [318]:
# Get all pairs of nodes at distance 2
potential_recommendations, potential_recommendation_dict = find_nodes_at_distance_2(training_graph, source_nodes)

In [319]:
# Include ground_truth in potential recommendations
potential_recommendations = potential_recommendations.union(ground_truth)
for e in ground_truth:
    if e[0] in potential_recommendation_dict:
        # We use sets to avoid including an edge that might be already contained in the distance 2 by chance
        potential_recommendation_dict[e[0]].append(e[1])
        potential_recommendation_dict[e[0]] = list(set(potential_recommendation_dict[e[0]]))
    else:
        potential_recommendation_dict[e[0]] = [e[1]]

# Link recommendations
We will generate recomendations for each source node in the test set. These recommendations will come from nodes at distance 2 and the test set.

We use 4 different techniques:
* Adamic-Adar similarity
* Personalized PageRank
* ALS algorithm
* Jaccard similarity

# Adamic-Adar

We need to compute the similiarity of each node in test set with all nodes at distance 2 in the graph. Then select the top-10 similar nodes.

In [264]:
def compute_ADA(u,v, graph):
    """
    Compute adamic-adar similarity for u and v 
    """

    # set of neighbors of u
    outlinks_from_u = graph.neighbors(u, mode='out')

    # set of neighbors of v
    inlinks_to_v =  graph.neighbors(v, mode='in')

    # set Z of neighbors of both
    bridges = set(outlinks_from_u).intersection(set(inlinks_to_v))

    # degree of nodes in set Z
    deg_ = graph.degree(list(bridges))
    
    # computing the reciprocal in log-scale
    out = [1./np.log2(dd+1) for dd in deg_]

    return sum(out)

In [265]:
import pandas as pd

total = len(potential_recommendation_dict.keys())

def compute_ada_for_source(source, dest, graph, df):
    """
    Compute adamic-adar similarity for source and all destinations 
    """
    
    # set of neighbors of u
    outlinks_from_source = graph.neighbors(source, mode='all')

    for v in dest:
        # set of neighbors of v
        inlinks_to_v =  graph.neighbors(v, mode='all')

        # set Z of neighbors of both
        bridges = set(outlinks_from_source).intersection(set(inlinks_to_v))

        # degree of nodes in set Z
        deg_ = graph.degree(list(bridges))

        # computing the reciprocal in log-scale
        out = [1./np.log2(dd+1) for dd in deg_]

        # Include in dataframe
        df = df.append({'source': source, 'dest': v, 'rating': sum(out)}, ignore_index=True)

    return df


def get_ada_recommendations(potential_recommendations, graph, df):
    """
    Computes the similarity for all candidate pairs of recommendations and returns a dataframe with the information.
    Also prints the state of the process each 100 samples.
    """
    count = 0

    for source in potential_recommendations:

        count+=1
        if count%100 == 0:
            print("==> {}/{}: {}%".format(count, total, count/total*100))

        dest = potential_recommendations[source]

        df = compute_ada_for_source(source, dest, graph, df)
  
    return df

In [266]:
# Execute ADA recommendation
df_ada = pd.DataFrame(columns=['source', 'dest', 'rating'])
df_ada = get_ada_recommendations(potential_recommendation_dict, training_graph, df_ada)

==> 100/4824: 2.0729684908789388%
==> 200/4824: 4.1459369817578775%
==> 300/4824: 6.218905472636816%
==> 400/4824: 8.291873963515755%
==> 500/4824: 10.364842454394694%
==> 600/4824: 12.437810945273633%
==> 700/4824: 14.510779436152571%
==> 800/4824: 16.58374792703151%
==> 900/4824: 18.65671641791045%
==> 1000/4824: 20.729684908789388%
==> 1100/4824: 22.802653399668323%
==> 1200/4824: 24.875621890547265%
==> 1300/4824: 26.9485903814262%
==> 1400/4824: 29.021558872305143%
==> 1500/4824: 31.094527363184078%
==> 1600/4824: 33.16749585406302%
==> 1700/4824: 35.240464344941955%
==> 1800/4824: 37.3134328358209%
==> 1900/4824: 39.38640132669983%
==> 2000/4824: 41.459369817578775%
==> 2100/4824: 43.53233830845771%
==> 2200/4824: 45.605306799336645%
==> 2300/4824: 47.67827529021559%
==> 2400/4824: 49.75124378109453%
==> 2500/4824: 51.824212271973465%
==> 2600/4824: 53.8971807628524%
==> 2700/4824: 55.970149253731336%
==> 2800/4824: 58.043117744610285%
==> 2900/4824: 60.11608623548922%
==> 3000/4

In [267]:
df_ada.head()

,source,dest,rating
0,1,51209,0
1,1,30734,0
2,1,15,11.5949
3,1,38927,0
4,1,16402,0


In [268]:
def obtain_topk(df, topk):
    """
    Returns a dataframe containing only the top-k recommendations for each source node
    """

    topk_df = pd.DataFrame(columns=['source', 'dest', 'rating', 'rank'])

    df = df.sort_values(by='rating', ascending=False)

    for source in df.source.unique():
        temp = df[df['source']==source]
        temp = temp.head(topk)[['source', 'dest', 'rating']]
        temp['rank'] = range(1, min(len(temp), topk) + 1)
        topk_df = topk_df.append(temp)

    return topk_df

In [269]:
# Store only top 10 recommendation for each source node
topk = 10
topk_ada = obtain_topk(df_ada, 10)

In [270]:
def is_in_groundtruth(row, ground_truth):
    """
    Includes the ground-truth information to the top-k recommendations.
    """
    edge = (int(row['source']), int(row['dest']))
    if edge in ground_truth:
        return 1
    else:
        return 0

def include_ground_truth(df, ground_truth):
    """
    For a dataframe and grountruth, includes whether a recommendation was in groundtruth
    """
    df['ground_truth'] = df.apply(lambda x: is_in_groundtruth(x, ground_truth), axis=1)
    return df

In [271]:
# Include groundtruth for ADA recommendations
df_ada_gt = include_ground_truth(topk_ada, ground_truth)

In [272]:
# How many correct predictions do we have
df_ada_gt[df_ada_gt['ground_truth']==1].shape

(4698, 5)

In [273]:
# Store recommendations
df_ada_gt.to_csv('ada_recommendations.csv')

# Personalized Pagerank

In [320]:
def compute_personalized_pagerank(u, graph, destinations=None, topk=10):
    """
    Compute personalized page-rank starting in node u. We can pass a list of destination nodes to filter only
    those of our interest.
    """
    # Compute personalized pagerank starting in node u
    pr = enumerate(graph.personalized_pagerank(reset_vertices=u, directed=False))
    
    # We must only consider those vertices in the potential recommendations (if applicable)
    if destinations:
        pr = [tup for tup in pr if tup[0] in destinations[u]]
        
    # Return topk sorted recommendations
    return sorted(pr, key=lambda tup: tup[1], reverse=True)[:topk]

In [321]:
# Compute personalized pagerank starting in each test node and querying only those at distance 2 from it
pagerank_df = pd.DataFrame(columns=['source', 'dest', 'rank', 'rating'])
topk = 10
count = 0
total = len(source_nodes)

for n in source_nodes:
    count+=1
    if count%100==0:
        print("==> {}/{}: {}%".format(count, total, 100*count/total))
    k = min(10, len(potential_recommendation_dict[n]))
    pr = compute_personalized_pagerank(n, training_graph, potential_recommendation_dict, k)
    temp = pd.DataFrame(pr, columns=['dest', 'rating'])
    temp['rank'] = range(1, k+1)
    temp['source'] = n
    pagerank_df = pagerank_df.append(temp)

==> 100/4865: 2.055498458376156%
==> 200/4865: 4.110996916752312%
==> 300/4865: 6.166495375128469%
==> 400/4865: 8.221993833504625%
==> 500/4865: 10.277492291880781%
==> 600/4865: 12.332990750256938%
==> 700/4865: 14.388489208633093%
==> 800/4865: 16.44398766700925%
==> 900/4865: 18.499486125385406%
==> 1000/4865: 20.554984583761563%
==> 1100/4865: 22.61048304213772%
==> 1200/4865: 24.665981500513876%
==> 1300/4865: 26.72147995889003%
==> 1400/4865: 28.776978417266186%
==> 1500/4865: 30.832476875642342%
==> 1600/4865: 32.8879753340185%
==> 1700/4865: 34.943473792394656%
==> 1800/4865: 36.99897225077081%
==> 1900/4865: 39.05447070914697%
==> 2000/4865: 41.109969167523126%
==> 2100/4865: 43.16546762589928%
==> 2200/4865: 45.22096608427544%
==> 2300/4865: 47.276464542651595%
==> 2400/4865: 49.33196300102775%
==> 2500/4865: 51.38746145940391%
==> 2600/4865: 53.44295991778006%
==> 2700/4865: 55.498458376156215%
==> 2800/4865: 57.55395683453237%
==> 2900/4865: 59.60945529290853%
==> 3000/486

In [322]:
# Include ground truth information
df_pagerank_gt = include_ground_truth(pagerank_df, ground_truth)

In [323]:
df_pagerank_gt.head()

,source,dest,rank,rating,ground_truth
0,1,133,1,0.013292,0
1,1,68,2,0.007337,0
2,1,15,3,0.005579,0
3,1,51,4,0.002039,0
4,1,21,5,0.001624,0


In [324]:
df_pagerank_gt[df_pagerank_gt['ground_truth']==1].shape

(4711, 5)

In [325]:
df_pagerank_gt.to_csv('pagerank_recommendations.csv')

# ALS

In [326]:
!pip install implicit

In [327]:
import implicit
from scipy.sparse import coo_matrix
from numpy import hstack, ones, array

def graph_to_sparse_matrix(graph):
    """
    Efficiently obtain sparse matrix for our graph.
    Using in-built method had complexity O(n^2) and couldn't be fitted in memory
    """
    xs, ys = map(array, zip(*graph.get_edgelist()))
    if not graph.is_directed():
        xs, ys = hstack((xs, ys)).T, hstack((ys, xs)).T
    else:
        xs, ys = xs.T, ys.T
    return coo_matrix((ones(xs.shape), (xs, ys)))

In [328]:
# Define ALS model
model = implicit.als.AlternatingLeastSquares(factors=10, calculate_training_loss=True,  iterations=5, use_gpu=False)

# Train the model on our sparse adjancecy matrix
model.fit(M)

In [329]:
def predict_ALS(df, potential_recommendations, model):
    """
    Predict for a list of source nodes the score for adding/removing a link to each of the potential recommendations
    """
    count = 0
    total = len(potential_recommendations.keys())

    # initialize the empty list
    all_predictions = []

    for source in potential_recommendations:

        count+=1
        if count%100==0:
            print("==> {}/{}: {}%".format(count, total, 100*count/total))

        array_n1 = model.user_factors[source, :]

        for dest in potential_recommendations[source]:
            array_n2 = model.item_factors[dest, :]

            # multiplying these vectors we generate an approximation for the edge score
            one_p = np.dot(array_n1, array_n2)

            df = df.append({'source': source, 'dest': dest, 'rating': one_p}, ignore_index=True)
        
    return df

In [331]:
# generate the predictions
df_als = pd.DataFrame(columns=['source', 'dest', 'rating'])
df_als = predict_ALS(df_als, potential_recommendation_dict, model)

==> 100/4865: 2.055498458376156%
==> 200/4865: 4.110996916752312%
==> 300/4865: 6.166495375128469%
==> 400/4865: 8.221993833504625%
==> 500/4865: 10.277492291880781%
==> 600/4865: 12.332990750256938%
==> 700/4865: 14.388489208633093%
==> 800/4865: 16.44398766700925%
==> 900/4865: 18.499486125385406%
==> 1000/4865: 20.554984583761563%
==> 1100/4865: 22.61048304213772%
==> 1200/4865: 24.665981500513876%
==> 1300/4865: 26.72147995889003%
==> 1400/4865: 28.776978417266186%
==> 1500/4865: 30.832476875642342%
==> 1600/4865: 32.8879753340185%
==> 1700/4865: 34.943473792394656%
==> 1800/4865: 36.99897225077081%
==> 1900/4865: 39.05447070914697%
==> 2000/4865: 41.109969167523126%
==> 2100/4865: 43.16546762589928%
==> 2200/4865: 45.22096608427544%
==> 2300/4865: 47.276464542651595%
==> 2400/4865: 49.33196300102775%
==> 2500/4865: 51.38746145940391%
==> 2600/4865: 53.44295991778006%
==> 2700/4865: 55.498458376156215%
==> 2800/4865: 57.55395683453237%
==> 2900/4865: 59.60945529290853%
==> 3000/486

In [332]:
# Store only top 10 recommendation for each source node
topk = 10
topk_als = obtain_topk(df_als, topk)

In [333]:
topk_als.reset_index(inplace=True, drop=True)

In [334]:
topk_als_gt = include_ground_truth(topk_als, ground_truth)

In [335]:
topk_als_gt[topk_als_gt['ground_truth']==1]

,source,dest,rating,rank,ground_truth
3,68.0,10767.0,7.097527e-02,4,1
5,68.0,51110.0,4.178395e-02,6,1
6,68.0,30891.0,4.028518e-02,7,1
7,68.0,49976.0,4.024546e-02,8,1
8,68.0,70990.0,4.024546e-02,9,1
...,...,...,...,...,...
21961,16512.0,16514.0,-2.526726e-13,1,1
21962,3217.0,63066.0,-8.620291e-13,1,1
21963,4348.0,4350.0,-4.409122e-12,1,1
21966,22089.0,22090.0,-7.727872e-12,1,1


In [336]:
topk_als_gt.to_csv('als_recommendations.csv')

# Jaccard

In [339]:
import pandas as pd

total = len(potential_recommendation_dict.keys())

def compute_Jaccard(source, dest, graph, df):
    """
    Compute Jaccard similarity for source and all potential destinations 
    """

    # set of neighbors of u
    outlinks_from_u = graph.neighbors(source, mode='out')

    for v in dest:

        # set of neighbors of v
        inlinks_to_v =  graph.neighbors(v, mode='in')

        # intesection of the two sets
        num = set(outlinks_from_u).intersection(set(inlinks_to_v))

        # union of the two sets
        den = set(outlinks_from_u).union(set(inlinks_to_v))

        # final division
        try:
            out = len(num)/len(den)
        except:
            out = 0

        # Include in dataframe
        df = df.append({'source': source, 'dest': v, 'rating': out}, ignore_index=True)
    
    return df


def get_jaccard_recommendations(potential_recommendations, graph, df):
    """
    Get jaccard recommendations for each source node and their potential destinations.
    """
    count = 0

    for source in potential_recommendations:

        count+=1
        if count%100 == 0:
            print("==> {}/{}: {}%".format(count, total, count/total*100))

        dest = potential_recommendations[source]

        df = compute_Jaccard(source, dest, graph, df)

    return df

In [340]:
df_jaccard = pd.DataFrame(columns=['source', 'dest', 'rating'])
df_jaccard = get_jaccard_recommendations(potential_recommendation_dict, training_graph, df_jaccard)

==> 100/4865: 2.055498458376156%
==> 200/4865: 4.110996916752312%
==> 300/4865: 6.166495375128469%
==> 400/4865: 8.221993833504625%
==> 500/4865: 10.277492291880781%
==> 600/4865: 12.332990750256938%
==> 700/4865: 14.388489208633093%
==> 800/4865: 16.44398766700925%
==> 900/4865: 18.499486125385406%
==> 1000/4865: 20.554984583761563%
==> 1100/4865: 22.61048304213772%
==> 1200/4865: 24.665981500513876%
==> 1300/4865: 26.721479958890033%
==> 1400/4865: 28.776978417266186%
==> 1500/4865: 30.832476875642346%
==> 1600/4865: 32.8879753340185%
==> 1700/4865: 34.943473792394656%
==> 1800/4865: 36.99897225077081%
==> 1900/4865: 39.05447070914697%
==> 2000/4865: 41.109969167523126%
==> 2100/4865: 43.16546762589928%
==> 2200/4865: 45.22096608427544%
==> 2300/4865: 47.27646454265159%
==> 2400/4865: 49.33196300102775%
==> 2500/4865: 51.3874614594039%
==> 2600/4865: 53.442959917780065%
==> 2700/4865: 55.49845837615622%
==> 2800/4865: 57.55395683453237%
==> 2900/4865: 59.60945529290853%
==> 3000/4865

In [341]:
# Store only top 10 recommendation for each source node
topk = 10
topk_jaccard = obtain_topk(df_jaccard, 10)

In [342]:
df_jaccard_gt = include_ground_truth(topk_jaccard, ground_truth)

In [343]:
df_jaccard_gt[df_jaccard_gt['ground_truth']==1].shape

(4688, 5)

In [344]:
df_jaccard_gt.to_csv('jaccard_recommendations.csv')

# RQ3A: Evaluation
The goal of this research question is evaluating our previous recommendation algorithms. The idea is the followiing:
* We have a training graph from which distance 1 edges have been deleted (tests-set)
* We will be recommending connections for nodes using distance 2 nodes and their distance 1 connections in the test set.
* A good recommendation systems should output those edges in the test set in the first positions since they are actually connected nodes.
* Following this idea, we will compute NDCG for our recommendations were test-set edges will be considered as good recommendations.
* Also accuracy will be computed as the number of test-edges among all top-k recommendations.

In [345]:
def dcg_at_k(y_true, y_score,  k=10):
    """
    Computes dcg@k for the true predictions and the one obtained with the model
    """
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)[:k]
    gain = 2**y_true - 1
    discounts = np.log2(np.arange(len(y_true))+2)
    
    return np.sum(gain / discounts)

def ndcg_at_k(y_true, y_score, k=10):
    """
    Computes the NDCG@K by comparing the ideal DCG with the obtained by the model
    """
    dcg_max = dcg_at_k(y_true, y_true, k)
    
    if not dcg_max:
        return 0
    
    return np.round(dcg_at_k(y_true, y_score, k)/dcg_max,4) 

def avg_ndcg_at_k(df, k=10):
    """
    Calculates the average NDCG@K
    """
    ndcg = []

    for source in df['source'].unique():
        temp = df[df['source']==source]
        y_score = temp['rating'].values
        y_true = temp['ground_truth'].values
        k_ = min(10, len(temp))
        ndcg.append(ndcg_at_k(y_true, y_score, k_))
    
    return np.mean(ndcg)

def compute_accuracy(df):
    """
    Computes accuracy for our dataframes. 
    Formula: Test-edges predicted/all recommendations.
    """
    return 100*len(df[df['ground_truth']==1])/len(df)

In [346]:
# Read previosly stored recommendations
df_jaccard_gt = pd.read_csv('jaccard_recommendations.csv', index_col=0)
df_ada_gt = pd.read_csv('ada_recommendations.csv', index_col=0)
df_als_gt = pd.read_csv('als_recommendations.csv', index_col=0)
df_personalized_pr_gt = pd.read_csv('pagerank_recommendations.csv', index_col=0)

In [347]:
# Compute average NDCG for all recommendation systems
k = 10
print("Average NDCG@{} for Jaccard = {}".format(k, avg_ndcg_at_k(df_jaccard_gt, k))) 
print("Average NDCG@{} for Adamic-Adar = {}".format(k, avg_ndcg_at_k(df_ada_gt, k))) 
print("Average NDCG@{} for ALS = {}".format(k, avg_ndcg_at_k(df_als_gt, k))) 
print("Average NDCG@{} for Personalized Pagerank = {}".format(k, avg_ndcg_at_k(df_personalized_pr_gt, k))) 

Average NDCG@10 for Jaccard = 0.6175270709146968
Average NDCG@10 for Adamic-Adar = 0.6199415008291874
Average NDCG@10 for ALS = 0.7471977595066802
Average NDCG@10 for Personalized Pagerank = 0.6213819527235355


In [348]:
# Compute accuracy for all recommendation systems
print("Accuracy for Jaccard = {}%".format(compute_accuracy(df_jaccard_gt))) 
print("Accuracy for Adamic-Adar = {}%".format(compute_accuracy(df_ada_gt))) 
print("Accuracy for ALS = {}%".format(compute_accuracy(df_als_gt))) 
print("Accuracy for Personalized Pagerank = {}%".format(compute_accuracy(df_personalized_pr_gt))) 

Accuracy for Jaccard = 21.3401310997815%
Accuracy for Adamic-Adar = 21.72385092018866%
Accuracy for ALS = 29.911689730517114%
Accuracy for Personalized Pagerank = 21.44482884195193%


# RQ3B: Improving the recommendation algorithms
Now, we will try to improve the accuracy of our recommendation algoritms. We use two different approaches:
* Weighted Personalized Pagerank: we use the destination node followers as edge weight.
* Bayesian Personalized Ranking: different algorithm to execute the predictions.

However, none of these strategies achieve better results than the best among the previous algorithms.

#### Weighted Personalized Pagerank

In [349]:
def compute_personalized_pagerank_weighted(u, graph, destinations=None, topk=10):
    """
    Computes weighted personalized pagerank from a node u
    """
    # Compute personalized pagerank starting in node u
    pr = enumerate(graph.personalized_pagerank(reset_vertices=u, directed=False, weights=graph.es['weight']))
    
    # We must only consider those vertices in the potential recommendations
    if destinations:
        pr = [tup for tup in pr if tup[0] in destinations[u]]
    
    # Return sorted pagerank recommendations
    return sorted(pr, key=lambda tup: tup[1], reverse=True)[:topk]

In [350]:
# Compute weighted personalized pagerank starting in each test node and querying only those at distance 2 from it
pagerank_df_weighted = pd.DataFrame(columns=['source', 'dest', 'rank', 'rating'])
topk = 10
count = 0
total = len(source_nodes)

for n in source_nodes:
    count+=1
    if count%100==0:
        print("==> {}/{}: {}%".format(count, total, 100*count/total))
    k = min(10, len(potential_recommendation_dict[n]))
    pr = compute_personalized_pagerank_weighted(n, w_training_graph, potential_recommendation_dict, k)
    temp = pd.DataFrame(pr, columns=['dest', 'rating'])
    temp['rank'] = range(1, k+1)
    temp['source'] = n
    pagerank_df_weighted = pagerank_df_weighted.append(temp)

==> 100/4865: 2.055498458376156%
==> 200/4865: 4.110996916752312%
==> 300/4865: 6.166495375128469%
==> 400/4865: 8.221993833504625%
==> 500/4865: 10.277492291880781%
==> 600/4865: 12.332990750256938%
==> 700/4865: 14.388489208633093%
==> 800/4865: 16.44398766700925%
==> 900/4865: 18.499486125385406%
==> 1000/4865: 20.554984583761563%
==> 1100/4865: 22.61048304213772%
==> 1200/4865: 24.665981500513876%
==> 1300/4865: 26.72147995889003%
==> 1400/4865: 28.776978417266186%
==> 1500/4865: 30.832476875642342%
==> 1600/4865: 32.8879753340185%
==> 1700/4865: 34.943473792394656%
==> 1800/4865: 36.99897225077081%
==> 1900/4865: 39.05447070914697%
==> 2000/4865: 41.109969167523126%
==> 2100/4865: 43.16546762589928%
==> 2200/4865: 45.22096608427544%
==> 2300/4865: 47.276464542651595%
==> 2400/4865: 49.33196300102775%
==> 2500/4865: 51.38746145940391%
==> 2600/4865: 53.44295991778006%
==> 2700/4865: 55.498458376156215%
==> 2800/4865: 57.55395683453237%
==> 2900/4865: 59.60945529290853%
==> 3000/486

In [351]:
df_pagerank_weighted_gt = include_ground_truth(pagerank_df_weighted, ground_truth)

In [352]:
df_pagerank_weighted_gt.head()

,source,dest,rank,rating,ground_truth
0,1,68,1,0.012958,0
1,1,133,2,0.007208,0
2,1,15,3,0.006726,0
3,1,21,4,0.005230,0
4,1,51,5,0.002985,0


In [353]:
df_pagerank_weighted_gt[df_pagerank_weighted_gt['ground_truth']==1].shape

(4735, 5)

In [354]:
df_pagerank_weighted_gt.to_csv('weighted_pagerank_recommendations.csv')

In [355]:
k = 10
print("Average NDCG@{} for Weighted Personalized Pagerank = {}".format(k, avg_ndcg_at_k(df_pagerank_weighted_gt, k))) 

Average NDCG@10 for Weighted Personalized Pagerank = 0.6216788900308325


In [356]:
print("Accuracy for Weighted Personalized Pagerank = {}%".format(compute_accuracy(df_pagerank_weighted_gt))) 

Accuracy for Weighted Personalized Pagerank = 21.5540786598689%


#### Bayesian Personalized Ranking

In [357]:
# Declare the BPR model
model_b = implicit.bpr.BayesianPersonalizedRanking(factors=20,  iterations=5, use_gpu=False)

# train the model on our sparse matrix
model_b.fit(M)

In [358]:
def predict_BPR(df, potential_recommendations, model):
    """
    Predict for a list of source nodes and their potential recommendations the score for adding/removing a link
    using Bayesian Personalized Ranking model.
    """
    count = 0
    total = len(potential_recommendations.keys())

    # initialize the empty list
    all_predictions = []

    for source in potential_recommendations:

        count+=1
        if count%100==0:
            print("==> {}/{}: {}%".format(count, total, 100*count/total))

        array_n1 = model.user_factors[source, :]

        for dest in potential_recommendations[source]:
            array_n2 = model.item_factors[dest, :]

            # multiplying these vectors we generate an approximation for the edge score
            one_p = np.dot(array_n1, array_n2)

            df = df.append({'source': source, 'dest': dest, 'rating': one_p}, ignore_index=True)
        
    return df

In [359]:
# Generate the predictions
df_bpr = pd.DataFrame(columns=['source', 'dest', 'rating'])
df_bpr = predict_BPR(df_bpr, potential_recommendation_dict, model_b)

==> 100/4865: 2.055498458376156%
==> 200/4865: 4.110996916752312%
==> 300/4865: 6.166495375128469%
==> 400/4865: 8.221993833504625%
==> 500/4865: 10.277492291880781%
==> 600/4865: 12.332990750256938%
==> 700/4865: 14.388489208633093%
==> 800/4865: 16.44398766700925%
==> 900/4865: 18.499486125385406%
==> 1000/4865: 20.554984583761563%
==> 1100/4865: 22.61048304213772%
==> 1200/4865: 24.665981500513876%
==> 1300/4865: 26.72147995889003%
==> 1400/4865: 28.776978417266186%
==> 1500/4865: 30.832476875642342%
==> 1600/4865: 32.8879753340185%
==> 1700/4865: 34.943473792394656%
==> 1800/4865: 36.99897225077081%
==> 1900/4865: 39.05447070914697%
==> 2000/4865: 41.109969167523126%
==> 2100/4865: 43.16546762589928%
==> 2200/4865: 45.22096608427544%
==> 2300/4865: 47.276464542651595%
==> 2400/4865: 49.33196300102775%
==> 2500/4865: 51.38746145940391%
==> 2600/4865: 53.44295991778006%
==> 2700/4865: 55.498458376156215%
==> 2800/4865: 57.55395683453237%
==> 2900/4865: 59.60945529290853%
==> 3000/486

In [360]:
# Store only top 10 recommendation for each source node
topk = 10
topk_bpr = obtain_topk(df_bpr, topk)

In [361]:
topk_bpr.reset_index(inplace=True, drop=True)

In [362]:
topk_bpr_gt = include_ground_truth(topk_bpr, ground_truth)

In [363]:
topk_bpr_gt[topk_bpr_gt['ground_truth']==1]

,source,dest,rating,rank,ground_truth
3,51.0,58144.0,0.042529,4,1
4,51.0,36180.0,0.040479,5,1
5,51.0,68918.0,0.040307,6,1
6,51.0,64496.0,0.039699,7,1
9,51.0,72661.0,0.033863,10,1
...,...,...,...,...,...
21963,5444.0,5445.0,-0.086376,1,1
21964,167.0,168.0,-0.091606,1,1
21965,6305.0,6306.0,-0.091839,1,1
21966,1156.0,1157.0,-0.119790,1,1


In [364]:
topk_bpr_gt.to_csv('bpr_recommendations.csv')

In [365]:
k = 10
print("Average NDCG@{} for Bayesian Personalized Pagerank = {}".format(k, avg_ndcg_at_k(topk_bpr_gt, k))) 

Average NDCG@10 for Bayesian Personalized Pagerank = 0.6975755395683453


In [366]:
print("Accuracy for Bayesian Personalized Pagerank = {}%".format(compute_accuracy(topk_bpr_gt))) 

Accuracy for Bayesian Personalized Pagerank = 26.006008739985432%
